# Using RAG to Build a Custom ChatBot
## 5. Production

> **Notice:**  
> Before starting this tutorial series, read up on the RAG pipeline.

This tutorial series assumes prerequisite understanding of RAG and therefore goes through the implementation of an advanced and customized RAG pipeline, explaining the micro-decisions made along the way.

> **Data Corpus:** 
> This tutorial uses [AGO yearly audit reports](https://www.ago.gov.sg/publications/annual-reports/) as an example. However, this repo's code is applicable to most pdf documents. The code examples for other documents (such as national day rally) will be referenced later. 

This notebook is the final tutorial in this tutorial series where we set up a chatbot for production. 

### Step 1: Chaining

At the moment, data is transferred from one jupyter cell to another. In production, this will have to all fit inside one python notebook. However, LangChain Expression Language ([LCEL](https://www.pinecone.io/learn/series/langchain/langchain-expression-language/)) is an elegant way of creating pipelines by chaining different processes together. 

LCEL is also an industry standard for custom LLM pipelines. I have provided multiple chains in the 2nd half of ["../utils/chatbot_utils.py"](../utils/chatbot_utils.py). 

I have also created a notebook, ["../notebooks/RAG_chaining.ipynb"](../notebooks/RAG_chaining.ipynb) that goes through the whole RAG pipeline but using chains. 


### Step 2: Agency - Deciding when to RAG

In production, the chatbot will be responding to user's multiple user inputs. However, not all user inputs require RAG. For example, greetings and request for summarising previous answers do not require RAG. Agents will have to therefore consider the purpose of the user's question as well as the conversation history to determine if RAG has to be performed on the latest user's query.

There are subtleties in creating an agent for this purpose. Should the chunks retrieved from previous user's questions be included in the conversation history fed into the agent? I performed experiments on this in ["../notebooks/agent_ReRAG.ipynb"](../notebooks/agent_ReRAG.ipynb). I found that passing only the bare conversation history to the agent without any inclusion of past retrieved chunks works just as well for AGO reports. 

Based on prompt engineering experiments, I developed 2 agents, ```need_RAG``` and ```all_agent``` that perform this task. ```need_RAG``` agent only focuses on deciding on when to RAG while ```all_agent``` performs this task along with others. The agent imporved its accuracy when we ask for reasoning for its decision prior to providing its decision. This is backed by research, known as *"chain of thought"*

Both agents and their prompts are included in ["../utils/agency.py"](../utils/agency.py).



### Step 3: Agency - Guard Rails

In order to prevent users from inputing prolematic inputs and the chatbot behaving undesirably, guard rails are required to be places at both the input and output of the LLM. [NVIDIA NeMo Guardrails](https://docs.nvidia.com/nemo/guardrails/index.html) offer multiple templates for agents that act as guardrails. 

I have created some guardrails using NeMo's Colang language in ```./config``` directory. I also experimented with these guardrails in ["../notebooks/guardrails.ipynb"](../notebooks/guardrails.ipynb).

However, I did not use these specific models in my final chatbot as they do not sync well with the langchain pipeline (demonstrated in the guardrails notebook). There are workarounds to force them into a langchain pipeline which I have provided in ["../utils/chatbot_utils.py"](../utils/chatbot_utils.py). These wordarounds however do not allow LLM streaming (to be explained next). 


### Step 4: Frontend & Backend

A modern chatbot has these few fundemental features apart from those mentioned previously
-  Remember and continue a conversation 
- Store, delete and add whole conversations
- Authentication
- Edit and copy inputs
- Regenerate outputs
- Streaming

I will show 3 versions of (frontend, backend), each of increasing complexity and features. The final version will be the chatbot that I used. 

> **Streaming:**  
> LLMs can take up to 20 seconds to produce a long responses. LLM streaming is the process of delivering output in real time, as each token is generated, rather than waiting for the entire response to complete. This enables users to receive information incrementally, providing quicker feedback and a more interactive experience.

### Version 1: Gradio Chatbot

[Gradio](https://www.gradio.app/guides/creating-a-chatbot-fast) is a python library that provides functionality to build both frontend and backend with a few lines of code. This is a great way to build a mock up to test the functionality of the RAG pipeline. However, it is only able to hold one conversation. Check out the code for my gradio app at ["../gradio_chatbot.py"](../gradio_chatbot.py)

Type ```python gradio_chatbot.py``` into the terminal/CLI to run the app.

### Version 2: Flask Backend

[Flask](https://flask.palletsprojects.com/en/3.0.x/quickstart/#a-minimal-application) is a popular backend server framework. However, LLM applications often require customised backends to allow for streaming, guardrails and other agents which are not offered by Flask. Therefore, flask backends for chatbots will have to be custom built to support these features. 

The advantage of Flask is that we can add additional components and components as we like. For example, if instead of streaming by tokens, we would like to stream by sentences, that is possible with Flask. I have built a Flask backend in ["../flask_app.py"](../flask_app.py) with the necessary chatbot support for your reference.

Type ```flask flask_app.py``` into the terminal/CLI to run the app.

### Version 3: LangServe Backend

[LangServe](https://blog.langchain.dev/introducing-langserve/), similar to Flask, is a backend server framework. However, it is built for LLM applications. It allows easy integration with LCEL chains and automatic streaming (various types of streaming available). Adding agents is also made simple.

The LangServe backend is built on top of FastAPI so it comes with swagger API. Swagger is a tool for documenting API endpoints. It also allows for testing all API calls in one place with all formats given so there is no need to use postman. 

[SWAGGER API](https://swagger.io/blog/api-documentation/what-is-api-documentation-and-why-it-matters/) can be accessed by adding a ```/docs``` to the end of the backend route. 

Checkout my LangServe app at ["../langserve_app.py"](../langserve_app.py).

Type ```python langserve_app.py``` into the terminal/CLI to run the app.

### React Frontend 
I modified [TurboGPT](https://github.com/mikebpech/turbogpt.ai)'s source code to create a frontend for out chatbot. It is available in the AuditBot_frontend Repo. It contains all the features mentioned before. Instructions on starting the frontend server is given in the [AuditBot_frontend](https://github.com/RaghaRao314159/AuditBot_frontend) Repo. 

This frontend contains code that requests from both a Flask server and a LangServe server. However, the langServe server is enabled by default. 